In [1]:
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.9 MB/s eta 0:00:00


In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
URL='https://arxiv.org/list/cs.AI/recent'


In [4]:
response = requests.get(URL)

In [5]:
html_content=response.text
#html_content

In [6]:
soup = BeautifulSoup(html_content, 'html.parser')


In [7]:
articles_section = soup.find('dl', id='articles')


In [8]:
# Extract PDF links
if articles_section:
    pdf_links = []
    for entry in articles_section.find_all('a', href=True):
        if entry['href'].startswith('/pdf/'):
            pdf_link = f"https://arxiv.org{entry['href']}"  # Construct full link
            pdf_links.append(pdf_link)

In [9]:
pdf_links

['https://arxiv.org/pdf/2410.14584',
 'https://arxiv.org/pdf/2410.14582',
 'https://arxiv.org/pdf/2410.14571',
 'https://arxiv.org/pdf/2410.14544',
 'https://arxiv.org/pdf/2410.14516',
 'https://arxiv.org/pdf/2410.14371',
 'https://arxiv.org/pdf/2410.14368',
 'https://arxiv.org/pdf/2410.14255',
 'https://arxiv.org/pdf/2410.14251',
 'https://arxiv.org/pdf/2410.14219',
 'https://arxiv.org/pdf/2410.14150',
 'https://arxiv.org/pdf/2410.14146',
 'https://arxiv.org/pdf/2410.13957',
 'https://arxiv.org/pdf/2410.13948',
 'https://arxiv.org/pdf/2410.14676',
 'https://arxiv.org/pdf/2410.14675',
 'https://arxiv.org/pdf/2410.14672',
 'https://arxiv.org/pdf/2410.14666',
 'https://arxiv.org/pdf/2410.14665',
 'https://arxiv.org/pdf/2410.14651',
 'https://arxiv.org/pdf/2410.14641',
 'https://arxiv.org/pdf/2410.14635',
 'https://arxiv.org/pdf/2410.14630',
 'https://arxiv.org/pdf/2410.14627',
 'https://arxiv.org/pdf/2410.14616',
 'https://arxiv.org/pdf/2410.14615',
 'https://arxiv.org/pdf/2410.14606',
 

In [10]:
soup = BeautifulSoup(html_content, 'html.parser')

In [11]:
papers = []
for entry in soup.find_all('div', class_='list-title'):
        title = entry.text.strip().replace("Title:", "").strip()
        #link = entry.find_previous_sibling('div', class_='list-identifier').a['href']
        #link = f"https://arxiv.org{link}.pdf"  # Direct PDF link
        papers.append(title)

In [12]:
papers_with_links = list(zip(papers, pdf_links))

In [13]:
pdf_paths = []


In [14]:
for title, pdf_link in papers_with_links:
    response = requests.get(pdf_link)

    # Save the PDF with the title as the filename
    filename = f"{title}.pdf"
    with open(filename, 'wb') as f:
        f.write(response.content)
        pdf_paths.append(filename)

In [15]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 53.8 MB/s eta 0:00:00


In [16]:
import pdfplumber

In [17]:
text=""

In [18]:
for pdf_path in pdf_paths:
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text()

**Till here, our web crawler is prepared, it has scraped the webpage provided, stored the links in a list , downloaded all the pdfs through that download links. Also, now it has extracted the text from the pdfs and storerd it in a list named as text.**

Now we will start with text preprocessing, this part involves the cleaning of the dataset, here we will be applying various techniques like lemitization, stemming, stopwords removal, Part of speech tagging and tokenization

In [19]:

from nltk.stem import WordNetLemmatizer

In [20]:
lemmatizer=WordNetLemmatizer()

In [21]:
from nltk.corpus import stopwords

In [22]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [23]:
stopword = stopwords.words('english')

In [24]:
stopword # these are the stopwords which we are going to remove from our dataset to refine it

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [25]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [26]:
sentences = nltk.sent_tokenize(text)

In [27]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [28]:
for i in range (len(sentences)):
  words=nltk.word_tokenize(sentences[i])
  for j in range (len(words)):
    if words[j] not in stopword:
      words[j]=lemmatizer.lemmatize(words[j])
  sentences[i]=' '.join(words)

Till now our dataset of text is cleaned by removing stopwords and lemmatizing the remaing words. Futher from here i have to encode my texts after which finally i can use it to do fine tuning of GPT or BERT on dataset which i have prepared.

After this it involves the step of, converting out text to vectors using Word2vec library on our dataset, Word2Vec library is developed by google and it is trained using google news dataset. It used Artificial neural network to train to convert the words to vectors. In its training the vectors were changed to minimize the loss function by foward propagation and back propagation algorithms as we commonly do in ANN.

**But now we will simply use a pre-trained model GPT-2 to predict the text. We will first do the fine-tuning of gpt so that it can generate text customized as per our dataset**

In [29]:
joined_text = ' '.join(sentences)

In [30]:
from transformers import GPT2Tokenizer


tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Example text (replace with your actual text)


# Tokenize the text
tokens = tokenizer.encode(joined_text)

# GPT-2 maximum sequence length
max_length = 1024

# Split tokens into chunks of max_length (1024 tokens)
token_chunks = [tokens[i:i + max_length] for i in range(0, len(tokens), max_length)]




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (924352 > 1024). Running this sequence through the model will result in indexing errors


In [31]:
import json

# Function to save tokenized text chunks to a file
def save_tokenized_chunks(tokenized_chunks, filename="tokenized_data.json"):
    with open(filename, "w") as file:
        json.dump(tokenized_chunks, file)

# Example usage
save_tokenized_chunks(token_chunks)

In [35]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
import torch

# Load GPT-2 model and tokenizer
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Check if a GPU is available and use it
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Move model to GPU if available

# Freeze all layers except the last transformer block
for param in model.parameters():
    param.requires_grad = False  # Freeze all parameters

# Unfreeze the last n layers of GPT-2 (you can adjust n)
n_unfreeze = 2  # Example: unfreezing the last 2 transformer blocks
for i in range(-n_unfreeze, 0):  # Unfreeze the last n blocks
    for param in model.transformer.h[i].parameters():
        param.requires_grad = True

# Unfreeze the lm_head (output layer) for training
for param in model.lm_head.parameters():
    param.requires_grad = True

# Load your dataset and tokenizer
texts = nltk.sent_tokenize(joined_text)  # Replace this with your list of texts

tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Resize the token embeddings because we added a new special token
model.resize_token_embeddings(len(tokenizer))

# Define tokenize function
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=1024)

# Tokenize your dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# Define the shift_labels function for next-word prediction
def shift_labels(examples):
    examples["labels"] = examples["input_ids"].copy()
    return examples

tokenized_dataset = tokenized_dataset.map(shift_labels, batched=True)

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=4,  # Adjust batch size based on GPU memory
    save_steps=10_000,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=100,
    report_to="none",  # Disable W&B or TensorBoard for simplicity
    fp16=True,  # Enable mixed precision training for faster training on GPUs
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# Train the model (this will use the GPU if available)
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/24699 [00:00<?, ? examples/s]

Map:   0%|          | 0/24699 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss
100,1.238900
200,0.251700
300,0.219900
400,0.228000
500,0.206200
600,0.211400
700,0.193300
800,0.174700
900,0.193200
1000,0.197000


TrainOutput(global_step=6175, training_loss=0.19729762135246987, metrics={'train_runtime': 2627.4873, 'train_samples_per_second': 9.4, 'train_steps_per_second': 2.35, 'total_flos': 1.2907303796736e+16, 'train_loss': 0.19729762135246987, 'epoch': 1.0})

In [36]:
model.save_pretrained("fine_tuned_gpt2")

In [50]:
input_text = "this is the finest research paper on machine learning i have ever seen"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)
attention_mask = torch.ones(input_ids.shape, device=device)  # Create attention mask manually

output = model.generate(input_ids,attention_mask=attention_mask, max_length=100, num_return_sequences=1, no_repeat_ngram_size=2)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
generated_text=nltk.sent_tokenize(generated_text)



print(generated_text[0:2])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['this is the finest research paper on machine learning i have ever seen, and have been used to find the best method for finding the optimal algorithm..The best algorithm is(1 ).', 'The best approach is.']
